Most of the post at PyBroMo's wiki work in current setting.
When the controller and engines are running on different hosts,
things are slightly more complicated, but the underlying ideas   
are the same:

1 Setup the controller
Create an IPython profile (ex: remote)
ipython profile create --parallel --profile=remote
Now, each time we want to start a parallel computation,  
we begin by starting the controller:
ipcontroller --profile=remote --ip=140.117.35.xxx

where the address is the controller ip address.

This command creates a file ipcontroller-engine.json that   
contains the connection info that the other machines need in  
order to connect to the controller. The file is located in   
IPYTHONDIR/profile_remote/security.

(In fact, we could use the following command to start 3   
engines and create the needed information to the file   
ipcontroller-engine.json.)

ipcluster start -n 3 --profile=remote --ip=140.117.35.xxx

We need to copy ipcontroller-engine.json to the slave machines
after the slave machine has been setted up.

2 Setup the "slave" machine Create an IPython profile   
(ex: remote) at slave machine.

ipython profile create --parallel --profile=remote

Paste ipcontroller-engine.json under   
IPYTHONDIR/profile_remote/security at the slave machine

We can start a computation engine with the ipengine command

ipengine --profile=remote

Or you can use the following way to provide the file directly.

ipengine --profile=remote --file=C:IPYTHONDIR/profile_remote/security/ipcontroller-engine.json

3 Create as many slave machines as you wish as in step 2.

4 Request Client as below from the cluster "remote".

In [1]:
import ipyparallel as ipp
import numpy as np
c = ipp.Client(profile='remote')
print(c.ids)

[0, 1, 2, 5]


In [2]:
dview = c[:]

We have now 4 engines!   
Make sure you paste rank_mpi.py to remote machine.

In [3]:
dview.run('rank_mpi.py')

<AsyncResult: execute>

In [4]:
%px totalhost = hostname()

In [5]:
dview['totalhost']

['DESKTOP-10711', 'DESKTOP-10711', 'DESKTOP-10711', 'LAPTOP-213RFLLM']

In [6]:
dview.scatter('a', range(10))
dview['a']

[range(0, 3), range(3, 6), range(6, 8), range(8, 10)]